In [1]:
import numpy as np
import os
import pickle
import sys
from PIL import Image
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

In [2]:
def load_cifar_batch(file_path):
    with open(file_path, 'rb') as f:
        if sys.version_info.major > 3:
            data = pickle.load(f, encoding='latin1')
        else:
            data = pickle.load(f, encoding='bytes')
        X = np.array(data[b'data']).reshape(-1, 32, 32, 3)  # Reshape the image data
        y = np.array(data[b'labels'])
        return X, y

In [5]:
def load_cifar10_dataset(dataset_dir):
    X_train = []
    y_train = []
    for batch_id in range(1, 6):
        file_path = os.path.join(dataset_dir, f'data_batch_{batch_id}')
        X, y = load_cifar_batch(file_path)
        X_train.append(X)
        y_train.append(y)
    
    X_train = np.concatenate(X_train, axis=0)
    y_train = np.concatenate(y_train, axis=0)
    
    file_path = os.path.join(dataset_dir, 'test_batch')
    X_test, y_test = load_cifar_batch(file_path)
    
    return X_train, y_train, X_test, y_test

In [7]:
# Set the directory stored the CIFAR-10 dataset
dataset_dir = r'C:\Users\rakib\Python project\cifar-10-batches-py'

In [8]:
# Load the CIFAR-10 dataset
X_train, y_train, X_test, y_test = load_cifar10_dataset(dataset_dir)

In [10]:
# Normalize pixel values
X_train = X_train / 255.0
X_test = X_test / 255.0

In [11]:
# Convert labels to one-hot encoding
num_classes = 10  # CIFAR-10 has 10 classes
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

In [12]:
# Split the dataset into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [13]:
# Create the model
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(32, 32, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

In [14]:
# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [15]:
# Train the model
batch_size = 128
epochs = 20
history = model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(X_val, y_val))

Epoch 1/20
313/313 [==============================] - 27s 83ms/step - loss: 1.8871 - accuracy: 0.3081 - val_loss: 1.5550 - val_accuracy: 0.4401
Epoch 2/20
313/313 [==============================] - 26s 84ms/step - loss: 1.5751 - accuracy: 0.4329 - val_loss: 1.4101 - val_accuracy: 0.4918
Epoch 3/20
313/313 [==============================] - 26s 84ms/step - loss: 1.4395 - accuracy: 0.4873 - val_loss: 1.3162 - val_accuracy: 0.5311
Epoch 4/20
313/313 [==============================] - 24s 76ms/step - loss: 1.3608 - accuracy: 0.5169 - val_loss: 1.2477 - val_accuracy: 0.5481
Epoch 5/20
313/313 [==============================] - 24s 75ms/step - loss: 1.2773 - accuracy: 0.5474 - val_loss: 1.2119 - val_accuracy: 0.5674
Epoch 6/20
313/313 [==============================] - 25s 79ms/step - loss: 1.2187 - accuracy: 0.5727 - val_loss: 1.1357 - val_accuracy: 0.5938
Epoch 7/20
313/313 [==============================] - 24s 77ms/step - loss: 1.1609 - accuracy: 0.5946 - val_loss: 1.1251 - val_accuracy:

In [16]:
# Save the model
model.save(r'C:\Users\rakib\Python project\model')

In [17]:
# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)

In [18]:
# Make predictions on the test set
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)

313/313 [==============================] - 2s 5ms/step


In [19]:
# Convert one-hot encoded labels to categorical labels
y_test_classes = np.argmax(y_test, axis=1)

In [22]:
# Calculate metrics
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
accuracy = accuracy_score(y_test_classes, y_pred_classes)
precision = precision_score(y_test_classes, y_pred_classes, average='weighted')
recall = recall_score(y_test_classes, y_pred_classes, average='weighted')
f1 = f1_score(y_test_classes, y_pred_classes, average='weighted')
print('Test Loss:', test_loss)
print('Test Accuracy:', test_accuracy)
print('Precision:', precision)
print('Recall:', recall)
print('F1-Score:', f1)

Test Loss: 1.1173354387283325
Test Accuracy: 0.6409000158309937
Precision: 0.6535682922756593
Recall: 0.6409
F1-Score: 0.6434936235921728
